# 📘 Clase 25: CRUD Completo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/heldigard/unaula-IF0100-POO-II/blob/main/notebooks/unidad-03/clase-25-crud-completo.ipynb)

## 🎯 Objetivos de Aprendizaje

Al finalizar esta clase, seras capaz de:
- Implementar CRUD completo con FastAPI
- Manejar relaciones en endpoints
- Implementar filtros y busquedas
- Manejar errores HTTP apropiadamente
- Crear endpoints anidados

---

## 📚 Schemas Completos

In [ ]:
# ============================================
# SCHEMAS COMPLETOS
# ============================================

from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime
from enum import Enum

class TaskStatus(str, Enum):
    PENDING = 'pendiente'
    IN_PROGRESS = 'en_progreso'
    COMPLETED = 'completada'

class UserBase(BaseModel):
    email: str
    username: str = Field(..., min_length=3, max_length=50)

class UserCreate(UserBase):
    password: str = Field(..., min_length=8)

class User(UserBase):
    id: int
    is_active: bool
    created_at: datetime
    
    class Config:
        orm_mode = True

class TaskBase(BaseModel):
    title: str = Field(..., min_length=1, max_length=200)
    description: Optional[str] = None
    status: TaskStatus = TaskStatus.PENDING

class TaskCreate(TaskBase):
    pass

class TaskUpdate(BaseModel):
    title: Optional[str] = None
    description: Optional[str] = None
    status: Optional[TaskStatus] = None

class Task(TaskBase):
    id: int
    project_id: int
    created_at: datetime
    
    class Config:
        orm_mode = True

print('✅ Schemas definidos')

---

## 🔌 Endpoints CRUD

In [ ]:
# ============================================
# ENDPOINTS COMPLETOS
# ============================================

ENDPOINTS_CODE = '''
from fastapi import APIRouter, Depends, HTTPException, status, Query
from sqlalchemy.orm import Session
from typing import List, Optional

router = APIRouter()

# READ con filtros
@router.get('/', response_model=List[Task])
def read_tasks(
    db: Session = Depends(get_db),
    skip: int = 0,
    limit: int = 100,
    status: Optional[TaskStatus] = None,
    search: Optional[str] = None
):
    query = db.query(TaskModel)
    
    if status:
        query = query.filter(TaskModel.status == status)
    
    if search:
        query = query.filter(TaskModel.title.contains(search))
    
    return query.offset(skip).limit(limit).all()

# CREATE
@router.post('/', response_model=Task, status_code=status.HTTP_201_CREATED)
def create_task(task_in: TaskCreate, db: Session = Depends(get_db)):
    db_task = TaskModel(**task_in.dict())
    db.add(db_task)
    db.commit()
    db.refresh(db_task)
    return db_task

# READ one
@router.get('/{task_id}', response_model=Task)
def read_task(task_id: int, db: Session = Depends(get_db)):
    task = db.query(TaskModel).filter(TaskModel.id == task_id).first()
    if not task:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail='Task not found'
        )
    return task

# UPDATE
@router.put('/{task_id}', response_model=Task)
def update_task(
    task_id: int,
    task_update: TaskUpdate,
    db: Session = Depends(get_db)
):
    task = db.query(TaskModel).filter(TaskModel.id == task_id).first()
    if not task:
        raise HTTPException(status_code=404, detail='Task not found')
    
    update_data = task_update.dict(exclude_unset=True)
    for field, value in update_data.items():
        setattr(task, field, value)
    
    db.commit()
    db.refresh(task)
    return task

# DELETE
@router.delete('/{task_id}', status_code=status.HTTP_204_NO_CONTENT)
def delete_task(task_id: int, db: Session = Depends(get_db)):
    task = db.query(TaskModel).filter(TaskModel.id == task_id).first()
    if not task:
        raise HTTPException(status_code=404, detail='Task not found')
    
    db.delete(task)
    db.commit()
    return None
'''

print(ENDPOINTS_CODE)

---

**¡Tu API REST esta lista! 🚀**